In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import datetime as dt
import csv

In [2]:
hoje = dt.datetime.today().strftime('%Y-%m-%d')

Subindo as bases

In [3]:
df_basetotal = pd.read_excel('input_sem_exclusao08042024_teste.xlsx', sheet_name='base_total')
df_recuperi = pd.read_excel('Base_camila_0204.xlsx', header=1)
df_acaocontrole = pd.read_csv('base_grupo_acao_controle.csv')

Volumetria da base total

In [4]:
df_basetotal.shape

(57483, 63)

Flag Protesto

In [5]:
df_recuperi.rename(columns={'Número do título': 'loan_id'}, inplace=True)
df_recuperi['flag_protesto'] = 1

In [6]:
df_recuperi = df_recuperi.loc[:,['loan_id','flag_protesto']]

In [7]:
df = pd.merge(df_basetotal,df_recuperi, how='left', on='loan_id')

In [8]:
df.fillna({'flag_protesto': 0}, inplace=True)
df['flag_protesto'] = df['flag_protesto'].astype(int)

In [9]:
df.groupby('flag_protesto')['loan_id'].count()

flag_protesto
0    48498
1     8985
Name: loan_id, dtype: int64

In [10]:
df_basetotal.shape

(57483, 63)

Criando demais flags e populando

In [11]:
df['flag_parcela'] = ''
df['estado'] = ''
df['regiao'] = ''
df['custo'] = ''
df['grupo_controle'] = ''
df['acao_controle'] = ''
df['retorno'] = ''

Flag Parcela

In [12]:
df[['loan_id','valor_parcela','flag_parcela']].head(5)

,loan_id,valor_parcela,flag_parcela
0,23488783,179.85,
1,19853726,422.48,
2,23896506,535.04,
3,22407538,133.24,
4,23901789,280.35,


In [13]:
for indice, linha in df.iterrows():
    if linha['valor_parcela'] > 300:
        df.at[indice, 'flag_parcela'] = '>300'
    else:
        df.at[indice, 'flag_parcela'] = '=<300'

In [14]:
df[['loan_id','valor_parcela','flag_parcela']].head(5)

,loan_id,valor_parcela,flag_parcela
0,23488783,179.85,=<300
1,19853726,422.48,>300
2,23896506,535.04,>300
3,22407538,133.24,=<300
4,23901789,280.35,=<300


Flag estado

In [15]:
df['estado'] = df['address_state']
#df['regiao'].fillna('na', inplace=True)
df.fillna({'estado': 0}, inplace=True)

In [16]:
df.groupby('estado')['loan_id'].count()

estado
0        10
AC      192
AL      353
AM      847
AP      204
BA     1994
CE     1073
DF     1655
ES     1126
GO     3068
MA      792
MG     5246
MS     1175
MT     1364
PA     1895
PB      535
PE     1051
PI      538
PR     3694
RJ     4141
RN      534
RO      539
RR      166
RS     3444
SC     2793
SE      273
SP    18331
TO      450
Name: loan_id, dtype: int64

Flag custo

In [17]:
for indice, linha in df.iterrows():
    if linha['estado'] == 'SP':
        df.at[indice, 'custo'] = 3.19
    else:
        df.at[indice, 'custo'] = 9.14

In [18]:
df[['loan_id','valor_parcela','estado','custo']].head(5)

,loan_id,valor_parcela,estado,custo
0,23488783,179.85,SE,9.14
1,19853726,422.48,SP,3.19
2,23896506,535.04,PA,9.14
3,22407538,133.24,CE,9.14
4,23901789,280.35,RJ,9.14


Flag Regiao

In [19]:
mapeamento_regioes = {
    'AC': 'Norte',
    'AL': 'Nordeste',
    'AP': 'Norte',
    'AM': 'Norte',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'DF': 'Centro-Oeste',
    'ES': 'Sudeste',
    'GO': 'Centro-Oeste',
    'MA': 'Nordeste',
    'MT': 'Centro-Oeste',
    'MS': 'Centro-Oeste',
    'MG': 'Sudeste',
    'PA': 'Norte',
    'PB': 'Nordeste',
    'PR': 'Sul',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'RJ': 'Sudeste',
    'RN': 'Nordeste',
    'RS': 'Sul',
    'RO': 'Norte',
    'RR': 'Norte',
    'SC': 'Sul',
    'SP': 'Sudeste',
    'SE': 'Nordeste',
    'TO': 'Norte'
}

df['regiao'] = df['estado'].map(mapeamento_regioes)

In [20]:
df[['loan_id','estado','regiao']].head(5)

,loan_id,estado,regiao
0,23488783,SE,Nordeste
1,19853726,SP,Sudeste
2,23896506,PA,Norte
3,22407538,CE,Nordeste
4,23901789,RJ,Sudeste


Flag grupo controle

In [21]:
df_acaocontrole.shape

(12942, 11)

In [22]:
df_grupocontrole = df_acaocontrole.query("data_retorno > @hoje")

In [23]:
df_grupocontrole.shape

(900, 11)

In [24]:
df_grupocontrole['data_inicio_protesto'] = pd.to_datetime(df_grupocontrole['data_inicio_protesto'])

df_grupocontrole['diferenca_dias'] = (dt.datetime.today() - df_grupocontrole['data_inicio_protesto']).dt.days

C:\Users\bagat\AppData\Local\Temp\ipykernel_14860\2171134396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grupocontrole['data_inicio_protesto'] = pd.to_datetime(df_grupocontrole['data_inicio_protesto'])
C:\Users\bagat\AppData\Local\Temp\ipykernel_14860\2171134396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grupocontrole['diferenca_dias'] = (dt.datetime.today() - df_grupocontrole['data_inicio_protesto']).dt.days


In [25]:
df_grupocontrole[['loan_id','diferenca_dias']]

,loan_id,diferenca_dias
12042,13671480,29
12043,26756869,29
12044,24800071,29
12045,23456816,29
12046,23810069,29
...,...,...
12937,24727221,29
12938,24816841,29
12939,15654848,29
12940,18649915,29


In [26]:
df_grupocontrole1 = df_grupocontrole.sort_values(by='diferenca_dias', ascending=True)

df_grupocontrole1 = df_grupocontrole1.drop_duplicates(subset='loan_id')

In [27]:
df_grupocontrole1[['loan_id','diferenca_dias']]

,loan_id,diferenca_dias
12042,13671480,29
12635,24211213,29
12636,24864139,29
12637,24500174,29
12638,24834695,29
...,...,...
12346,24654173,29
12347,25902440,29
12348,25986768,29
12378,20390880,29


In [28]:
df_grupocontrole1.drop(columns='diferenca_dias', inplace=True)

In [29]:
df_grupocontrole1.shape

(900, 11)

In [30]:
df_grupocontrole1['flag_controle'] = '1'

In [31]:
df_grupocontrole1 = df_grupocontrole1.loc[:,['loan_id','flag_controle','grupo']]

In [32]:
df.shape

(57483, 71)

In [33]:
df = pd.merge(df,df_grupocontrole1,how='left', on='loan_id')

In [34]:
df.shape

(57483, 73)

In [35]:
df.head(5)

,dt_ref,borrower_id,loan_id,CPF,lecca_id,reference_date,safra,safra_reparc,conditions,cluster,...,flag_protesto,flag_parcela,estado,regiao,custo,grupo_controle,acao_controle,retorno,flag_controle,grupo
0,202404,4955621,23488783,3690846595,9632570.0,2024-04-01,2023-08-01,-,original,demais,...,0,=<300,SE,Nordeste,9.14,,,,NaN,NaN
1,202404,3730893,19853726,21785286862,4675629.0,2024-04-01,2023-03-01,-,original,pagou_adjusted_value,...,1,>300,SP,Sudeste,3.19,,,,1,acao
2,202404,3261294,23896506,60350554358,10744377.0,2024-04-01,2023-08-01,-,original,demais,...,1,>300,PA,Norte,9.14,,,,NaN,NaN
3,202404,8643792,22407538,77023315349,7852121.0,2024-04-01,2023-06-01,2023-11,reparcelamento,demais,...,0,=<300,CE,Nordeste,9.14,,,,NaN,NaN
4,202404,7682675,23901789,14082808760,10690598.0,2024-04-01,2023-08-01,-,original,demais,...,0,=<300,RJ,Sudeste,9.14,,,,NaN,NaN


In [36]:
#df['flag_controle'].fillna(0, inplace=True)
df.fillna({'flag_controle': 0}, inplace=True)
df['flag_controle'] = df['flag_controle'].astype(int)

df.fillna({'grupo': 0}, inplace=True)

In [37]:
df.groupby('flag_controle')['loan_id'].count()

flag_controle
0    56634
1      849
Name: loan_id, dtype: int64

In [38]:
df['grupo_controle'] = df['flag_controle']
df['acao_controle'] = df['grupo']

In [39]:
df = df.drop(['flag_controle','grupo'], axis=1)

Flag retorno

In [40]:
df_retorno = df_acaocontrole.query("data_retorno <= @hoje")

In [41]:
df_retorno.shape

(12042, 11)

In [42]:
df_retorno['data_inicio_protesto'] = pd.to_datetime(df_retorno['data_inicio_protesto'])

df_retorno['diferenca_dias'] = (dt.datetime.today() - df_retorno['data_inicio_protesto']).dt.days

C:\Users\bagat\AppData\Local\Temp\ipykernel_14860\2913227487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retorno['data_inicio_protesto'] = pd.to_datetime(df_retorno['data_inicio_protesto'])
C:\Users\bagat\AppData\Local\Temp\ipykernel_14860\2913227487.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retorno['diferenca_dias'] = (dt.datetime.today() - df_retorno['data_inicio_protesto']).dt.days


In [43]:
df_retorno.head()

,loan_id,borrower_id,late_category_expandida_noverde_atual,estado,regiao,semana,data_inicio_protesto,data_retorno,grupo,custo_protesto,mot_fidc,diferenca_dias
0,15937649,2971790,3 - 60a90,SP,SUDESTE,semana 01,2024-01-09,2024-02-16,controle,3.84,"2360,31",102
1,6711419,2814124,3 - 60a90,BA,NORDESTE,semana 01,2024-01-09,2024-02-16,controle,9.14,"695,85",102
2,23267037,9334658,3 - 60a90,SP,SUDESTE,semana 01,2024-01-09,2024-02-16,acao,3.84,"496,47",102
3,18017258,205021,3 - 60a90,SP,SUDESTE,semana 01,2024-01-09,2024-02-16,controle,3.84,"1640,86",102
4,22838906,6031564,3 - 60a90,SP,SUDESTE,semana 01,2024-01-09,2024-02-16,controle,3.84,"669,83",102


In [44]:
df_retorno = df_retorno.sort_values(by='diferenca_dias', ascending=True)

In [45]:
df_retorno.head()

,loan_id,borrower_id,late_category_expandida_noverde_atual,estado,regiao,semana,data_inicio_protesto,data_retorno,grupo,custo_protesto,mot_fidc,diferenca_dias
12041,20413380,8391895,5 - 120a150,SP,SUDESTE,semana 08,2024-03-15,2024-04-15,controle,"3,84","6566,39",36
10767,26315348,2598884,3 - 60a90,GO,CENTRO-OESTE,semana 08,2024-03-15,2024-04-15,acao,"9,14","1166,79",36
10768,26581829,12110391,3 - 60a90,PR,SUL,semana 08,2024-03-15,2024-04-15,controle,"9,14","936,15",36
10769,26898353,12218607,3 - 60a90,SP,SUDESTE,semana 08,2024-03-15,2024-04-15,acao,"3,84","1303,14",36
10770,26378631,12054315,3 - 60a90,SP,SUDESTE,semana 08,2024-03-15,2024-04-15,controle,"3,84","911,79",36


In [46]:
df_retorno = df_retorno.drop_duplicates(subset='loan_id')

df_retorno.drop(columns='diferenca_dias', inplace=True)

In [47]:
df_retorno.shape

(9131, 11)

In [48]:
df_retorno.head()

,loan_id,borrower_id,late_category_expandida_noverde_atual,estado,regiao,semana,data_inicio_protesto,data_retorno,grupo,custo_protesto,mot_fidc
12041,20413380,8391895,5 - 120a150,SP,SUDESTE,semana 08,2024-03-15,2024-04-15,controle,"3,84","6566,39"
10767,26315348,2598884,3 - 60a90,GO,CENTRO-OESTE,semana 08,2024-03-15,2024-04-15,acao,"9,14","1166,79"
10768,26581829,12110391,3 - 60a90,PR,SUL,semana 08,2024-03-15,2024-04-15,controle,"9,14","936,15"
10769,26898353,12218607,3 - 60a90,SP,SUDESTE,semana 08,2024-03-15,2024-04-15,acao,"3,84","1303,14"
10770,26378631,12054315,3 - 60a90,SP,SUDESTE,semana 08,2024-03-15,2024-04-15,controle,"3,84","911,79"


In [49]:
df_retorno = df_retorno.loc[:,['loan_id','data_retorno','semana']]

In [50]:
df_retorno.head(3)

,loan_id,data_retorno,semana
12041,20413380,2024-04-15,semana 08
10767,26315348,2024-04-15,semana 08
10768,26581829,2024-04-15,semana 08


In [51]:
df.shape

(57483, 71)

In [52]:
df = pd.merge(df,df_retorno, how='left', on='loan_id')

In [53]:
df.shape

(57483, 73)

In [54]:
df['retorno'] = df['semana']
df['retorno'].fillna(0, inplace=True)

In [55]:
df = df.drop(['data_retorno','semana'], axis=1)

Filtrando faixas de atraso

In [56]:
atraso = ['3 - 60a90', '4 - 90a120', '5 - 120a150']
df_final = df[df['late_category_expandida_noverde_atual'].isin(atraso)]

In [57]:
df_final.shape

(21158, 71)

TabDin volumetria

In [58]:
volumetria = df_final.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='flag_protesto', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

volumetria

flag_protesto,0,1,Total
late_category_expandida_noverde_atual,,,
3 - 60a90,7460,924,8384
4 - 90a120,5118,1601,6719
5 - 120a150,4310,1745,6055
Total,16888,4270,21158


TabDin grupo controle

In [59]:
df_final.groupby('grupo_controle')['loan_id'].count()

grupo_controle
0    20313
1      845
Name: loan_id, dtype: int64

In [60]:
acao_controle = df_final.query("grupo_controle == 1")

In [61]:
flag_acaocontrole = acao_controle.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='acao_controle', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

flag_acaocontrole

acao_controle,acao,controle,Total
late_category_expandida_noverde_atual,,,
3 - 60a90,396,412,808
4 - 90a120,25,8,33
5 - 120a150,3,1,4
Total,424,421,845


TabDin protesto

In [62]:
protesto = df_final.query("grupo_controle == 0")

In [63]:
flag_protesto = protesto.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='flag_protesto', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

flag_protesto

flag_protesto,0,1,Total
late_category_expandida_noverde_atual,,,
3 - 60a90,7046,530,7576
4 - 90a120,5108,1578,6686
5 - 120a150,4309,1742,6051
Total,16463,3850,20313


TabDin acordo a vencer

In [64]:
acordoavencer = df_final.query("grupo_controle ==0 & flag_protesto==0")

In [65]:
flag_acordo = acordoavencer.pivot_table(index='late_category_expandida_noverde_atual', 
                             columns='acordo_a_vencer', 
                             values='loan_id', 
                             aggfunc='count', 
                             margins=True, 
                             margins_name='Total')

flag_acordo

acordo_a_vencer,0,1,Total
late_category_expandida_noverde_atual,,,
3 - 60a90,6868,178,7046
4 - 90a120,5031,77,5108
5 - 120a150,4260,49,4309
Total,16159,304,16463


TabDin flag parcela

In [66]:
parcela = df_final.query("grupo_controle == 0 & flag_protesto == 0 & acordo_a_vencer == 0")

In [67]:
flag_parcela = parcela.pivot_table(index='late_category_expandida_noverde_atual',
                                   columns='flag_parcela',
                                   values='loan_id',
                                   aggfunc='count',
                                   margins=True,
                                   margins_name='Total')
flag_parcela

flag_parcela,=<300,>300,Total
late_category_expandida_noverde_atual,,,
3 - 60a90,5076,1792,6868
4 - 90a120,4095,936,5031
5 - 120a150,3628,632,4260
Total,12799,3360,16159


TabDin semana

In [68]:
semana = df_final.query("grupo_controle == 0 & flag_protesto == 0 & acordo_a_vencer == 0 & flag_parcela == '>300' & retorno !=0")

In [69]:
flag_semana = semana.pivot_table(index='late_category_expandida_noverde_atual',
                                 columns='retorno',
                                 values='loan_id',
                                 aggfunc='count',
                                 margins=True,
                                 margins_name='Total')
flag_semana

retorno,semana 01,semana 02,semana 03,semana 04,semana 05,semana 06,semana 07,semana 08,Total
late_category_expandida_noverde_atual,,,,,,,,,
3 - 60a90,NaN,NaN,1.0,4.0,15.0,21.0,70.0,626.0,737
4 - 90a120,1.0,NaN,NaN,NaN,2.0,562.0,206.0,164.0,935
5 - 120a150,1.0,1.0,NaN,1.0,NaN,137.0,392.0,98.0,630
Total,2.0,1.0,1.0,5.0,17.0,720.0,668.0,888.0,2302


TabDin regiao

In [70]:
reg = df_final.query("grupo_controle == 0 & flag_protesto == 0 & acordo_a_vencer == 0 & flag_parcela == '>300'")

In [71]:
flag_regiao = reg.pivot_table(index='late_category_expandida_noverde_atual',
                              columns='regiao',
                              values='loan_id',
                              aggfunc='count',
                              margins=True,
                              margins_name='Total')
flag_regiao

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul,Total
late_category_expandida_noverde_atual,,,,,,
3 - 60a90,231,169,110,953,329,1792
4 - 90a120,124,95,53,490,174,936
5 - 120a150,95,59,31,340,107,632
Total,450,323,194,1783,610,3360


Tabela Final

In [76]:
faixa_atraso = df_final['late_category_expandida_noverde_atual']
total = df_final.groupby(faixa_atraso)['loan_id'].count()
acao = df_final.query("acao_controle =='acao' & grupo_controle == 1").groupby(faixa_atraso)['loan_id'].count()
controle = df_final.query("acao_controle =='controle' & grupo_controle == 1").groupby(faixa_atraso)['loan_id'].count()
protestado = df_final.query("flag_protesto == 1 & grupo_controle == 0").groupby(faixa_atraso)['loan_id'].count()
acordovencer = df_final.query("grupo_controle == 0 & flag_protesto == 0 & acordo_a_vencer == 1").groupby(faixa_atraso)['loan_id'].count()
trezentos = df_final.query("grupo_controle == 0 & flag_protesto == 0 & acordo_a_vencer == 0 & flag_parcela == '=<300'").groupby(faixa_atraso)['loan_id'].count()

In [77]:
disponivel = total - acao - controle - protestado - acordovencer - trezentos

In [82]:
df_tabela = pd.DataFrame({
    'Total': total,
    'Acao': acao,
    'Controle': controle,
    'Protesto': protestado,
    'Acordo a Vencer': acordovencer,
    'Valor Parcela <=300': trezentos,
    'Disponivel': disponivel
})

df_tabela = df_tabela.fillna(0)

df_tabela

,Total,Acao,Controle,Protesto,Acordo a Vencer,Valor Parcela <=300,Disponivel
late_category_expandida_noverde_atual,,,,,,,
3 - 60a90,8384,396,412,530,178,5076,1792
4 - 90a120,6719,25,8,1578,77,4095,936
5 - 120a150,6055,3,1,1742,49,3628,632


Publico selecionado

In [79]:
df_publico = df_final.query("grupo_controle == 0 & flag_protesto == 0 & acordo_a_vencer == 0 & flag_parcela =='>300'")

In [80]:
df_publico.shape

(3360, 71)

In [84]:
contagem_regioes = df_publico['regiao'].value_counts()

df_acao = pd.DataFrame(columns=df_publico.columns)
df_controle = pd.DataFrame(columns=df_publico.columns)

for regiao, contagem in contagem_regioes.items():
    metade = (contagem + 1) // 2
    
    indices_acao = df_publico[df_publico['regiao'] == regiao].sample(n=metade, replace=False).index
    
    try:
        indices_controle = df_publico[df_publico['regiao'] == regiao].drop(indices_acao).sample(n=metade, replace=False).index
    except ValueError:
        indices_controle = df_publico[df_publico['regiao'] == regiao].drop(indices_acao).sample(n=contagem - metade, replace=False).index
    

    df_acao = pd.concat([df_acao, df_publico.loc[indices_acao]])
    df_controle = pd.concat([df_controle, df_publico.loc[indices_controle]])

assert len(df_acao) == len(df_acao['loan_id'].unique())
assert len(df_controle) == len(df_controle['loan_id'].unique())

C:\Users\bagat\AppData\Local\Temp\ipykernel_14860\2228877962.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_acao = pd.concat([df_acao, df_publico.loc[indices_acao]])
C:\Users\bagat\AppData\Local\Temp\ipykernel_14860\2228877962.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_controle = pd.concat([df_controle, df_publico.loc[indices_controle]])


In [86]:
df_acao.shape

(1681, 71)

In [88]:
df_controle.shape

(1679, 71)

In [89]:
df_acao.groupby('late_category_expandida_noverde_atual')['loan_id'].count()

late_category_expandida_noverde_atual
3 - 60a90      905
4 - 90a120     466
5 - 120a150    310
Name: loan_id, dtype: int64

In [90]:
df_controle.groupby('late_category_expandida_noverde_atual')['loan_id'].count()

late_category_expandida_noverde_atual
3 - 60a90      887
4 - 90a120     470
5 - 120a150    322
Name: loan_id, dtype: int64

In [94]:
df_cpf = df_acao.copy()
df_cpf = df_cpf.loc[:,['CPF']]
df_cpf['CPF'] = "'" + df_cpf['CPF'].astype(str) + "',"
df_cpf.head(3)

,CPF
50591,"'24999713802',"
34279,"'31281198803',"
43194,"'9077053751',"


In [95]:
df_cpf.to_excel(f'Clientes disponiveis para protesto {hoje}.xlsx', index=False)